# 使用 MongoDB整理 OpenStreetMap 数据

## 1. 在地图中遇到的问题

+ 邮编数据有非6位数字数据
+ 名称' name' 中文、英文还有其他语言混杂，且 name有多个不同tag 标识
+ 有些路名仅英文，没有中文，中文相同发音有多条道路，没办法确定实际名称
    - node 574755937，addr:street': 'East Zhaohua Road
+ tag的key和内容不一致
    - 如tag key是city，但实际内容可能是区域名称，或带有门牌号的地址。
+ 旅游特色，如animal，动物名称，直接被写成地点的名称。
+ 不同的人对街道，地址的理解不同，比如说‘弄’，有些理解位街道，放到路名中，有些理解为门牌号。
+ district理解也有不同，有理解为行政区域，也有理解为通俗概念上的区域，这些通俗概念的区域可能比行政区域大或小，比如‘浦西’比district要大。
+ subway 表示，不符合OSM规范


## 2. 数据概述

### 原始数据

+ 数据来源 : mapzen.com
    - [下载页面](https://mapzen.com/data/metro-extracts/metro/shanghai_china/102027181/Shanghai/)
    - [下载链接](https://s3.amazonaws.com/metro-extracts.mapzen.com/shanghai_china.osm.bz2)
+ 文件大小
     - 871274652 12月  6 10:58 shanghai_china.osm
+ 原始node记录
    - 原始node记录数
        + `db.org_osm.count({'type': 'node'})`
        + 4141878
    - 原始空node数
        + 空记录仅包含创建人信息和坐标信息，没有其他信息，没有实际含义，在数据整理前删除
        + `db.org_osm.count({'type': 'node','tags': {$exists: false}})`
        + 3959968
+ 原始way记录
    - 原始way记录数
        + `db.org_osm.count({'type': 'way'})`
        + 490805
    - 原始空way记录数
        + 空道路不包含节点，在数据整理前删除
        + `db.org_osm.count({'type': 'way', 'nd_refs': {$exists: false}})`
        + 744

### 整理前后的数据

整理前数目即原始记录去除空记录

+ node数目
    - 整理前 181910
    - 整理后 27959
+ way 数目
    - 整理前 490061
    - 整理后 490061

### 整理后数据

+ 用户数
    - `db.osm_collection.distinct("created.uid").length`
    - 2325
+ 贡献最大用户
    - `db.osm_collection.aggregate([{$group:{"_id":"$created.user", "count":{$sum:1}}}, {$sort:{"count": -1}}, {$limit:1}])`
    - `{ "_id" : "Chen Jia", "count" : 94123 }`
+ 仅贡献一次用户数
    - `db.osm_collection.aggregate([{$group:{"_id":"$created.user", "count":{"$sum":1}}},  {$group:{"_id":"$count", "num_users":{$sum:1}}},{$sort:{"_id":1}}, {$limit:1}])`
    - `{ "_id" : 1, "num_users" : 739 }`
+ 不同shop 数量
    - `db.osm_collection.aggregate([{$match: {'shop': {$exists: true}}},{$group: {'_id': '$shop', 'count': {'$sum':1}}}, {$sort: {'count': -1}}])`
    - supermarket 最多 619
    - 其次 convenience，476


## 3.  关于数据集的其他想法


1. 在数据数据阶段加入验证，比如邮编，如果不是6位，不可以保存
2. 不同语言对应到确定的key下面, 如name:UN:en，统一为 name:en
3. 数据集中空数据太多，数据记录工具的显著位置，添加对空数据的说明，提示避免输入空数据。